In [2]:
import json
import glob
import os
import multiprocessing
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm

### Creating files with brands/models for collecting

In [159]:
test_df = pd.read_csv('data/test.csv')
test_df_modified = pd.read_csv('data/test_modified.csv')

In [160]:
test_df.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\r\nАвтопрага Север\r\nДанный авт...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\r\nАвтопрага Север\r\n\r\nВы мож...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [128]:
test_df['color'].value_counts()

чёрный         11010
белый           6133
серебристый     4113
серый           3984
синий           3289
красный         1531
коричневый      1501
зелёный          952
бежевый          639
голубой          457
золотистый       266
пурпурный        254
фиолетовый       219
жёлтый           194
оранжевый        131
розовый           13
Name: color, dtype: int64

In [97]:
all_brands = list(test_df['brand'].value_counts().index)
print(all_brands)

['BMW', 'VOLKSWAGEN', 'NISSAN', 'MERCEDES', 'TOYOTA', 'AUDI', 'MITSUBISHI', 'SKODA', 'VOLVO', 'HONDA', 'INFINITI', 'LEXUS']


In [105]:
# divide by odd and even (since they are in decreasing amount)
odd_brands = all_brands[1::2]
even_brands = all_brands[0::2]

odd_brands_models_dict = dict()
even_brands_models_dict = dict()

for brand in odd_brands:
    odd_brands_models_dict[brand] = list(test_df[test_df['brand'] == brand]['model_name'].unique())
    
for brand in even_brands:
    even_brands_models_dict[brand] = list(test_df[test_df['brand'] == brand]['model_name'].unique())

with open('data/odd_brands_models_dict.json', 'w') as file:
    json.dump(odd_brands_models_dict, file)
    
with open('data/even_brands_models_dict.json', 'w') as file:
    json.dump(even_brands_models_dict, file)

In [127]:
print(f"odd brands: {odd_brands}")
print(f"even brands: {even_brands}")

odd brands: ['VOLKSWAGEN', 'MERCEDES', 'AUDI', 'SKODA', 'HONDA', 'LEXUS']
even brands: ['BMW', 'NISSAN', 'TOYOTA', 'MITSUBISHI', 'VOLVO', 'INFINITI']


In [158]:
time_str = datetime.now().strftime('%Y_%m_%d-%H_%M_%S')

### Multiprocessing

In [54]:
def foo(name):
    print(f"my name is: {name}")
    sleep_time = round(random.random() * 5, 1)
    print(f"i am sleeping for {sleep_time}")
    time.sleep(sleep_time)
    print("i am awake")

In [55]:
p1 = multiprocessing.Process(target=foo, args=(all_brands[0],))
p2 = multiprocessing.Process(target=foo, args=(all_brands[1],))
p3 = multiprocessing.Process(target=foo, args=(all_brands[2],))
p4 = multiprocessing.Process(target=foo, args=(all_brands[3],))

In [56]:
p1.start()
p2.start()
p3.start()
p4.start()

my name is: SKODA
my name is: AUDIi am sleeping for 4.1

i am sleeping for 3.3my name is: HONDA

i am sleeping for 0.6
my name is: VOLVO
i am sleeping for 0.9
i am awake
i am awake
i am awake
i am awake


In [70]:
try:
    requests.get('https://github.com/', timeout=0.1)
except requests.exceptions.Timeout:
    print("timeout")
except requests.exceptions.RequestException:
    print("general request exception")

timeout


In [9]:
np.round(145042.43452)

145042.0